In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

C:\Users\asus\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\asus\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
imdb_df = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)

In [6]:
imdb_df.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [7]:
amazon_df = pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)

In [8]:
yelp_df = pd.read_csv('yelp_labelled.txt',sep='\t',header=None)

In [12]:
df = pd.DataFrame()

In [14]:
df = df.append(imdb_df)

In [15]:
df = df.append(amazon_df)
df = df.append(yelp_df)

In [17]:
df.shape

(2748, 2)

In [19]:
df.columns = ['Reviews','Sentiment']

In [20]:
df.head()

,Reviews,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [33]:
def textProcessing(df):
#     1. Tokenization
    tokens = []
    for i in range(len(df)):
        tokens.append(word_tokenize(df['Reviews'].iloc[i].lower()))
    
#     2. Remove Stopwords
    words = []
    eng_stopwords = stopwords.words('english')
    eng_stopwords.extend([',','?','!','@','.','-','&'])
    for tokenList in tokens:
        list_1 = []
        for i in range(len(tokenList)):
            if tokenList[i] not in eng_stopwords:
                list_1.append(tokenList[i])
        words.append(list_1)
    
#     3. Lemmatization
    wnet = WordNetLemmatizer()
    for i in range(len(words)):
        for j in range(len(words[i])):
            words[i][j] = wnet.lemmatize(words[i][j], pos='v')
            
    for i in range(len(words)):
        words[i] = ' '.join(words[i])
    
    return words
    

In [34]:
wordsList = textProcessing(df)

In [39]:
wordsList[100]

'go see movie bring pillow girlfriend/boyfriend keep occupy'

In [40]:
tfidf = TfidfVectorizer()

In [46]:
vector = tfidf.fit_transform(wordsList)
vector = vector.toarray()

In [47]:
y = df['Sentiment']
x_train,x_test,y_train,y_test = train_test_split(vector,y,test_size=0.25)

In [48]:
gauss_nb = GaussianNB()
gauss_nb.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [49]:
y_pred = gauss_nb.predict(x_test)
accuracy_score(y_test,y_pred)

0.6404657933042213

In [50]:
multi_nb = MultinomialNB()
multi_nb.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [51]:
y_pred = multi_nb.predict(x_test)
accuracy_score(y_test,y_pred)

0.8049490538573508

In [52]:
reg = LogisticRegression()
reg.fit(x_train,y_train)

C:\Users\asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [53]:
y_pred = reg.predict(x_test)
accuracy_score(y_test,y_pred)

0.7947598253275109

In [62]:
new_rev = "Hrithik and Tiger both are genetically gifted and they have worked hard on their bodies. Offcourse they had enough funds available to do that since they were born. But these are not enough to make a movie good, same old cliche, same old stunts. Seen thousand times in the past. This is an era when you can't fool the audience with nobrainers like these, action movies are good when action is supported with a nice plot or storyline. Nothing is in the movie, the movie will do well on the box office initially because of the heavy marketing and the appeal of the lead actors but it won't go further. A third grade movie which tries hard to be a poor copy of famous Hollywood action movies. Only villagers and autowallahs will love this."

In [63]:
test_df = pd.DataFrame({"Reviews":[new_rev]})
test_wordsList = textProcessing(test_df)

In [64]:
test_wordsList

["hrithik tiger genetically gift work hard body offcourse enough fund available since bear enough make movie good old cliche old stunt see thousand time past era ca n't fool audience nobrainers like action movies good action support nice plot storyline nothing movie movie well box office initially heavy market appeal lead actors wo n't go third grade movie try hard poor copy famous hollywood action movies villagers autowallahs love"]

In [65]:
test_vect = tfidf.transform(test_wordsList)

In [66]:
test_matrix = test_vect.toarray()

In [67]:
multi_nb.predict(test_matrix)

array([0], dtype=int64)

In [68]:
import pickle as pk

In [69]:
file = open('tfidf.pkl','wb')
pk.dump(tfidf,file)
file.close()

file = open('nb.pkl','wb')
pk.dump(multi_nb,file)
file.close()